In [13]:
import torch
from torchvision import transforms
from PIL import Image
import torchvision.models as models
import json

In [14]:
img = Image.open('/workspace/quant/sample_imgs/n01443537_goldfish.JPEG')

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
input_batch = input_batch.to(device)
input_batch

tensor([[[[-1.5014, -1.5185, -1.4843,  ..., -1.7583, -1.7412, -1.7069],
          [-1.4500, -1.5014, -1.4843,  ..., -1.7412, -1.7240, -1.6898],
          [-1.4500, -1.5185, -1.5185,  ..., -1.7069, -1.6898, -1.6727],
          ...,
          [-1.6384, -1.5699, -1.5528,  ..., -1.6727, -1.6384, -1.6384],
          [-1.6555, -1.6213, -1.5870,  ..., -1.6384, -1.6213, -1.6213],
          [-1.6555, -1.6213, -1.6213,  ..., -1.6213, -1.6213, -1.6213]],

         [[-1.3004, -1.2654, -1.2129,  ..., -1.6331, -1.6155, -1.5805],
          [-1.2304, -1.2304, -1.2129,  ..., -1.6155, -1.5980, -1.5630],
          [-1.1779, -1.2304, -1.2479,  ..., -1.5805, -1.5630, -1.5455],
          ...,
          [-1.4755, -1.3880, -1.3529,  ..., -1.5105, -1.4755, -1.4405],
          [-1.5105, -1.4405, -1.4055,  ..., -1.5105, -1.4580, -1.4230],
          [-1.5280, -1.4755, -1.4405,  ..., -1.5105, -1.4580, -1.4230]],

         [[-1.1421, -1.1596, -1.1247,  ..., -1.4210, -1.4036, -1.3687],
          [-1.0898, -1.1421, -

In [15]:
# 使用 TorchScript 加载模型
arch = 'squeezenet1_0'
scripted_model_name = f"/workspace/quant/models/quantized_{arch}_v1.pt"
loaded_scripted_model = torch.jit.load(scripted_model_name)
loaded_scripted_model.eval()
loaded_scripted_model.state_dict()

# Perform inference with the loaded quantized model
with torch.no_grad():
    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension
    loaded_output = loaded_scripted_model(input_batch)


In [16]:
# Find the maximum value and its index
loaded_max_value, loaded_max_index = torch.max(loaded_output, dim=1)
# Compare the maximum values and their indices
print("Loaded Max Value:", loaded_max_value)
print("Loaded Max Index:", loaded_max_index)

Loaded Max Value: tensor([30.7403])
Loaded Max Index: tensor([1])


In [17]:
# Load ImageNet labels
with open('imagenet-simple-labels.json') as f:
    labels = json.load(f)

# Print predicted class label
predicted_label = labels[loaded_max_index]
print(f'Predicted class: {loaded_max_index}, Label: {predicted_label}')

Predicted class: tensor([1]), Label: goldfish
